In [16]:
from cpc_pipeline import *

from llm import LLM
gpt35t = LLM("gpt-3.5-turbo")
gpt4 = LLM("gpt-4")

In [17]:
import nltk
import random

filtered_corpus = [s for s in nltk.corpus.abc.words() if s.lower() == s]
def make_caesar_cipher(word_length, shift):
    word = random.choice([s for s in filtered_corpus if len(s) == word_length])
    return ''.join([chr((ord(c) + shift - 97) % 26 + 97) for c in word]), word

problem_df = cpc_problems(make_caesar_cipher, {'word_length': [3,4,5], 'shift':range(1, 6)}, n=1, unwrap_colnames=['problem', 'original'])

Creating cpc_pipeline/make_caesar_cipher3286947891.csv...


In [18]:
prompt = "This ciphertext is either an anagram or a Caesar cipher. First, assume it's an anagram, and try to unscramble it. If that doesn't work, continue by assuming it's a Caesar cipher and try to decrypt it. Don't give up until you decrypt it! Ciphertext: {ciphertext}"
false_start = "Ok, I'll start by treating the ciphertext as an anagram and trying to unscramble it."

def solve_caesar_gpt35t(problem):
    return gpt35t.chat_completion_false_start(prompt.format(ciphertext=problem), false_start=false_start)

def solve_caesar_gpt4(problem):
    return gpt4.chat_completion_false_start(prompt.format(ciphertext=problem), false_start=false_start)

context_df_35t = cpc_contexts(problem_df, solve_caesar_gpt35t)
context_df_4 = cpc_contexts(problem_df, solve_caesar_gpt4)

Creating cpc_pipeline/solve_caesar_gpt35t4090895247.csv...
Creating cpc_pipeline/solve_caesar_gpt42184766945.csv...


In [19]:
context_df_35t['correct'] = context_df_35t.apply(lambda x: 1 if x['original'] in x['context'] else 0, axis=1)
print(np.mean(context_df_35t['correct']))
context_df_4['correct'] = context_df_4.apply(lambda x: 1 if x['original'] in x['context'] else 0, axis=1)
print(np.mean(context_df_4['correct']))

0.06666666666666667
0.4


In [20]:
print('\n-----------------\n'.join(context_df_35t['context']))

After trying different combinations, I couldn't unscramble the ciphertext "bsf". Now, I will try to decrypt it assuming it's a Caesar cipher with a shift of 1.
-----------------
After trying different combinations, I couldn't unscramble "vjg" into a meaningful word. I will now try to decrypt it using a Caesar cipher.
-----------------
After trying to unscramble "dqg", it doesn't seem to form any meaningful word. Let's try decrypting it using a Caesar cipher with a shift of 1. 

Input: dqg
Shift 1: erh

After decrypting with a Caesar cipher shift of 1, the plaintext is "erh".
-----------------
After trying different combinations, I couldn't unscramble the ciphertext. Now, I will try to decrypt it assuming it's a Caesar cipher. I will try shifting the letters by different numbers to see if I can make sense of the message.
-----------------
After trying different combinations, I couldn't find a meaningful word. Let me now try decrypting it using a Caesar cipher.
-----------------
After tr

In [ ]:
from new_switch_utils import make_non_decreasing

anagram_switching_prompt = "This is a reasoning transcript of an agent trying to decipher some ciphertext. The agent will start treating the ciphertext as an anagram and attempting to unscramble it. The agent may then give up and treat the ciphertext as a Caesar cipher instead. First you will be shown the full transcript, then just a prefix of the transcript. By the end of the prefix transcript, has the agent switched from treating the ciphertext as an anagram to treating it as a Caesar cipher yet?"
def judge_anagram_switching(context, prefix):
        return 1 if gpt4.yesno_completion(anagram_switching_prompt+'\n\nFULL TRANSCRIPT:\n'+context+'\n\nPREFIX TRANSCRIPT:\n'+prefix+"\n\nANSWER:\n") == 'Yes' else 0

def split_and_judge_gpt4(context, chunk_size):
    prefixes = [context[:pref_end] for pref_end in range(chunk_size, len(context)+chunk_size, chunk_size)]
    measured_switching = [judge_anagram_switching(context, prefix) for prefix in prefixes]
    processed_switching = random.choice(make_non_decreasing(measured_switching))
    return prefixes, processed_switching

switching_df_35t = split_and_judge_switching(context_df_35t, split_and_judge_gpt4, 50)
switching_df_4 = split_and_judge_switching(context_df_4, split_and_judge_gpt4, 50)

In [ ]:
anagram_cpc_prompt = """At this point, we're going to stop and consider whether this approach is working or not 
              and leads to a correct solution to the problem being worked on. 
              Otherwise, we should step back and try a different approach."""

from solver import perform_one_token_cpc, perform_cot_cpc
def gpt35t_1t(context):
    return perform_one_token_cpc(gpt35t, context, prompt=anagram_cpc_prompt)
def gpt35t_cot(context):
    return perform_cot_cpc(gpt35t, context, prompt=anagram_cpc_prompt)
def gpt4_1t(context):
    return perform_one_token_cpc(gpt4, context, prompt=anagram_cpc_prompt)
def gpt4_cot(context):
    return perform_cot_cpc(gpt4, context, prompt=anagram_cpc_prompt)

cpc_df_35t = judge_cpc(switching_df_35t, [gpt35t_1t, gpt35t_cot], radius_right=7)
cpc_df_4 = judge_cpc(switching_df_4, [gpt4_1t, gpt4_cot], radius_right=7)

In [ ]:
do_analysis(cpc_df_35t, [gpt35t_1t, gpt35t_cot])
do_analysis(cpc_df_4, [gpt4_1t, gpt4_cot])